In [29]:
## project 1

from google.colab import files

uploaded = files.upload()

Saving CreditBanking_Project1.xls to CreditBanking_Project1.xls


In [30]:
!pip install pandas openpyxl

import pandas as pd

# Load the Excel file with multiple sheets
xl = pd.ExcelFile('CreditBanking_Project1.xls')

# Read each sheet into a DataFrame
customer_df = xl.parse(0)
spend_df = xl.parse(1)
repayment_df = xl.parse(2)

# Handle missing values in each DataFrame
customer_df.fillna(0, inplace=True)  # Assuming missing values are replaced with 0 for numerical columns
spend_df.fillna(0, inplace=True)
repayment_df.fillna(0, inplace=True)

# Provide a meaningful treatment to all values where age is less than 18
customer_df.loc[customer_df['Age'] < 18, 'Age'] = 18  # Replace age < 18 with 18

spend_df['Month'] = pd.to_datetime(spend_df['Month'], format='%d-%b-%Y')
spend_df['month_year'] = spend_df['Month'].dt.strftime('%m-%Y')

repayment_df['Month'] = pd.to_datetime(repayment_df['Month'], format='%d-%b-%Y')
repayment_df['month_year'] = repayment_df['Month'].dt.strftime('%m-%Y')


In [31]:
# Is there any customer who has spent more than his/her Credit Limit for any particular month?
credit_limit_check = pd.merge(spend_df, customer_df, left_on='Costomer', right_on='Customer')
credit_limit_check = credit_limit_check[credit_limit_check['Amount'] > credit_limit_check['Limit']]
credit_limit_violations = credit_limit_check[['Costomer', 'Amount', 'Limit','month_year']]

In [37]:
# Monthly spend of each customer
monthly_spend = spend_df.groupby(['Costomer', 'month_year']).agg({'Amount': 'sum'}).reset_index()

# Monthly repayment of each customer
monthly_repayment = repayment_df.groupby(['Costomer', 'month_year']).agg({'Amount': 'sum'}).reset_index()

# Highest paying 10 customers
highest_paying_customers = monthly_spend.groupby('Costomer').agg({'Amount': 'sum'}).nlargest(10, 'Amount')
highest_paying_customers.reset_index(inplace=True)

# People in which segment are spending more money
segment_spending = pd.merge(customer_df, spend_df, left_on='Customer', right_on='Costomer')
segment_spending = segment_spending.groupby('Segment').agg({'Amount': 'sum'}).reset_index()

# Which age group is spending more money
age_group_spending = pd.merge(customer_df, spend_df, left_on='Customer', right_on='Costomer')
age_group_spending['Age_Group'] = pd.cut(age_group_spending['Age'], bins=[18, 25, 35, 45, float('inf')], labels=['18-25', '26-35', '36-45', '45+'])
age_group_spending = age_group_spending.groupby('Age_Group').agg({'Amount': 'sum'}).reset_index()

In [47]:
# Which is the most profitable segment

merged_df = pd.merge(spend_df, customer_df, left_on='Costomer', right_on='Customer')
most_profitable_segment = merged_df.groupby('Segment').agg({'Amount': 'sum'}).nlargest(1, 'Amount')
most_profitable_segment.reset_index(inplace=True)
print(most_profitable_segment)

         Segment        Amount
0  Normal Salary  1.077071e+08


In [68]:
# In which category the customers are spending more money
category_spending = pd.merge(spend_df, customer_df, left_on='Costomer', right_on='Customer')
category_spending = category_spending.groupby('Credit Card Product').agg({'Amount': 'sum'}).reset_index()

# Impose an interest rate of 2.9% for each customer for any due amount
repayment_df_with_interest = pd.merge(spend_df, repayment_df, left_on='Costomer', right_on='Costomer')
repayment_df_with_interest['due'] = repayment_df_with_interest['Amount_x']-repayment_df_with_interest['Amount_y']
interest_rate = 0.029
repayment_df_with_interest['Interest Amount'] = repayment_df_with_interest['due'] * interest_rate
repayment_df_with_interest['due_with_interest'] = repayment_df_with_interest['due'] + repayment_df_with_interest['Interest Amount']

# Monthly profit for the bank
total_repayment = spend_df.groupby('Month')['Amount'].sum()
total_interest_income = repayment_df.groupby('Month')['Amount'].sum()
monthly_profit = total_interest_income - total_repayment

In [35]:
!pip install xlsxwriter

In [69]:
import xlsxwriter

In [54]:
with pd.ExcelWriter('output1.xlsx', engine='xlsxwriter') as writer:
    credit_limit_violations.to_excel(writer, index=False, sheet_name='credit_limit_violations')
    monthly_spend.to_excel(writer, index=False, sheet_name='monthly_spend')
    monthly_repayment.to_excel(writer, index=False, sheet_name='monthly_repayment')
    highest_paying_customers.to_excel(writer, index=False, sheet_name='highest_paying_customers')
    segment_spending.to_excel(writer, index=False, sheet_name='segment_spending')
    age_group_spending.to_excel(writer, index=False, sheet_name='age_group_spending')
    most_profitable_segment.to_excel(writer, index=False, sheet_name='most_profitable_segment')
    category_spending.to_excel(writer, index=False, sheet_name='category_spending')
    repayment_df_with_interest.to_excel(writer, index=False, sheet_name='repayment_df_with_interest')
    monthly_profit.to_excel(writer, index=False, sheet_name='monthly_bank_profit')